In [1]:
import pandas as pd
from pyspark.sql import *

In [2]:
#A partir de la plataforma online (e-shop) de los países en los que opera, Nintendo tiene información 
#de ventas de videojuegos diarias digitales por país en el siguiente RDD: 
#(id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias). Por otro lado se tienen otro 
#RDD que tiene información de todos los videojuegos que se venden en su plataforma con el siguiente 
#formato (id_videojuego, titulo, rating_pegi, rating_esbr). Tener en cuenta que un mismo videojuego 
#se puede vender en distintos países y esos nos permitirá obtener métricas a nivel global. Con esta 
#información escribir un programa en pySpark que permita: 

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlc = SQLContext(sc)
ventas = sqlc.read.csv("ventas.csv").rdd
juegos = sqlc.read.csv("juegos.csv").rdd
ventas = ventas.map(lambda x:(pd.to_numeric(x[0],"coerce"), pd.to_numeric(x[1],"coerce"),\
                             pd.to_datetime(x[2],"coerce"),pd.to_numeric(x[3],"coerce"),\
                             pd.to_numeric(x[4],"coerce"))).cache()
juegos = juegos.map(lambda x:(pd.to_numeric(x[0],"coerce"),x[1],pd.to_numeric(x[2],"coerce"),\
                              pd.to_numeric(x[3],"coerce"))).cache()

In [3]:
#a) Obtener el videojuego con más ventas 
#digitales globales (es decir en todos los países) en un RDD con el siguiente formato: 
#(id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales

id_y_nombre = juegos.map(lambda x:(x[0],x[1]))#(id_juego,nombre)
id_y_ventas = ventas.map(lambda x:(x[0],x[4]))#(id,ventas diarias)
id_ventas_totales = id_y_ventas.reduceByKey(lambda x,y:x+y)
juego_mas_vendido = id_ventas_totales.reduce(lambda x,y: x if x[1]>y[1] else y)
rdd_juego_mas_vendido = sc.parallelize([juego_mas_vendido])
rdd_mas_vendido_final = rdd_juego_mas_vendido.leftOuterJoin(id_y_nombre).\
                                map(lambda x:(x[0],x[1][1],x[1][0])).cache()
rdd_mas_vendido_final.collect()

[(4, 'nwbal', 644600)]

In [5]:
#b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una 
#mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias)
id_mas_vendido = rdd_mas_vendido_final.first()[0]
ventas_mas_vendido_por_pais = ventas.filter(lambda x:x[0]==id_mas_vendido)
ventas_mas_vendido_por_pais.map(lambda x:(x[1],(x[4],x[3]))).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).\
            reduce(lambda x,y:x if x[1][0]/x[1][1]>y[1][0]/y[1][1] else y)

(4, (37899, 62505))